In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Masking, Dense, LSTM, GRU, SimpleRNN
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

2022-06-27 17:21:44.692751: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-27 17:21:44.692776: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# for a start, let's use the imdb reviews dataset
from tensorflow.keras.datasets import imdb

In [3]:
def load_data(percentage_of_sentences=None):
    # Load the data
    (sentences_train, y_train), (sentences_test, y_test) = imdb.load_data()
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(sentences_train))
        sentences_train = sentences_train[:len_train]
        y_train = y_train[:len_train]
        
        len_test = int(percentage_of_sentences/100*len(sentences_test))
        sentences_test = sentences_test[:len_test]
        y_test = y_test[:len_test]
            
    # Load the {interger: word} representation
    word_to_id = imdb.get_word_index()
    word_to_id = {k:(v+3) for k,v in word_to_id.items()}
    for i, w in enumerate(['<PAD>', '<START>', '<UNK>', '<UNUSED>']):
        word_to_id[w] = i

    id_to_word = {v:k for k, v in word_to_id.items()}

    # Convert the list of integers to list of words (str)
    X_train = [' '.join([id_to_word[_] for _ in sentence[1:]]) for sentence in sentences_train]
    
    return X_train


### Just run this cell to load the data
data = load_data(percentage_of_sentences=10)

In [4]:
def get_X_y(sentence, length=40):
    '''
    returns a tuple containing:
    - a sentence (a string)
    - the word immediately following that sentence
    '''
    words = sentence.split()
    # return None if the sentence is too short
    if len(words) <= length:
        return None
    # pick a random part of the sentence
    first_word_idx = np.random.randint(0, len(words) - length)
    # build X (a part of the sentence) and y (the word immediately following X)
    X = words[first_word_idx : first_word_idx + length]
    y = words[first_word_idx + length]
    # return X and y
    return X, y

In [5]:
# check the output of the function
get_X_y(data[0])

(['robert',
  "redford's",
  'is',
  'an',
  'amazing',
  'actor',
  'and',
  'now',
  'the',
  'same',
  'being',
  'director',
  "norman's",
  'father',
  'came',
  'from',
  'the',
  'same',
  'scottish',
  'island',
  'as',
  'myself',
  'so',
  'i',
  'loved',
  'the',
  'fact',
  'there',
  'was',
  'a',
  'real',
  'connection',
  'with',
  'this',
  'film',
  'the',
  'witty',
  'remarks',
  'throughout',
  'the'],
 'film')

In [6]:
def create_dataset(sentences, number_of_samples = 10000):
    '''
    creates a dataset (X and y) made of number_of_samples observations
    '''
    X, y = [], []
    indices = np.random.randint(0, len(sentences), size=number_of_samples)
    # call get_X_y number_of_samples times
    for idx in indices:
        ret = get_X_y(sentences[idx])
        if ret is None:
            continue
        xi, yi = ret
        X.append(xi)
        y.append(yi)
    return X, y

In [7]:
X, y = create_dataset(data)

In [8]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [9]:
len(X_train), len(X_test), len(y_train), len(y_test)

(6939, 2974, 6939, 2974)

In [10]:
# fit a word2vec model
word2vec = Word2Vec(sentences=X_train, vector_size=60, min_count=10, window=10)
wv = word2vec.wv

In [11]:
# check embedding size
wv["movie"].size

60

In [12]:
wv.most_similar("movie")

[('thing', 0.9436944127082825),
 ('film', 0.9328452348709106),
 ('worst', 0.8986397385597229),
 ('really', 0.8773297071456909),
 ("wasn't", 0.8706840872764587),
 ('worth', 0.8684008121490479),
 ('entertaining', 0.8641366362571716),
 ('ever', 0.8624119758605957),
 ('watching', 0.8595168590545654),
 ("isn't", 0.8518555760383606)]

In [13]:
wv["movie"]

array([ 0.26939258,  0.22727671,  1.6425986 , -0.46513474, -0.81536955,
        0.07606534,  0.52022386,  1.217694  , -0.95508873, -0.18229   ,
        2.0107858 , -0.01701614, -0.3375833 , -0.00599152, -0.38202333,
       -1.2598183 ,  0.6614996 ,  0.3316515 , -0.8055477 , -1.8089521 ,
       -0.444411  , -0.12864034,  1.1217287 ,  0.05814089, -0.02403587,
       -0.10075924,  0.04603994,  0.0330493 , -0.4630286 , -0.28347018,
       -1.0758255 ,  0.26436836,  0.369587  , -0.45169133, -0.15273842,
        0.01122016, -0.8233154 , -1.1630796 , -0.38601926, -0.64197487,
        0.29810837, -0.6036165 , -0.24657275, -0.07866705,  0.48859063,
        0.505661  , -0.9897906 ,  0.06472152,  0.0919769 ,  0.41475278,
        0.04984819, -0.02200857,  0.25209785, -0.23751783, -0.48187834,
        0.33497384,  0.87743294,  0.04533954,  0.41354102, -1.3827277 ],
      dtype=float32)

In [14]:
wv.similar_by_vector(wv["movie"])

[('movie', 0.9999998807907104),
 ('thing', 0.943694531917572),
 ('film', 0.9328452348709106),
 ('worst', 0.8986397385597229),
 ('really', 0.8773297071456909),
 ("wasn't", 0.8706841468811035),
 ('worth', 0.8684008121490479),
 ('entertaining', 0.8641366362571716),
 ('ever', 0.8624119758605957),
 ('watching', 0.8595169186592102)]

In [15]:
vocab_size = len(wv.key_to_index)
vocab_size

2589

In [16]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=40)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=40)

In [17]:
# check X_train_pad and W_test_pad
type(X_train_pad)

numpy.ndarray

In [18]:
X_train_pad.shape

(6939, 40, 60)

In [19]:
X_test_pad.shape

(2974, 40, 60)

In [20]:
wv.vector_size

60

In [21]:
len(X_train)

6939

In [22]:
len(X_test)

2974

In [23]:
# check y_train and y_test
type(y_train)

list

In [24]:
len(y_train)

6939

In [25]:
# check how many words from y_train are in wv
sum([word in wv for word in y_train]) / len(y_train)

0.8684248450785416

In [26]:
# check how many words from y_test are in wv
sum([word in wv for word in y_test]) / len(y_test)

0.8688634835238735

In [27]:
X_train_pad.shape, X_test_pad.shape, len(y_train), len(y_test)

((6939, 40, 60), (2974, 40, 60), 6939, 2974)

In [28]:
# filter X_train_pad (and X_test_pad) to keep only the observations for which y_train (and y_test) is in wv
mask_train = [word in wv for word in y_train]
X_train_pad = X_train_pad[mask_train, :, :]
y_train = [word for word in y_train if word in wv]
mask_test = [word in wv for word in y_test]
X_test_pad = X_test_pad[mask_test, :, :]
y_test = [word for word in y_test if word in wv]

In [29]:
X_train_pad.shape, X_test_pad.shape, len(y_train), len(y_test)

((6026, 40, 60), (2584, 40, 60), 6026, 2584)

In [30]:
# transform y_train and y_test into vectors
y_train_vec = np.array([wv[word] for word in y_train])
y_test_vec = np.array([wv[word] for word in y_test])

In [31]:
y_train_vec.shape, y_test_vec.shape

((6026, 60), (2584, 60))

In [32]:
y_train_vec[0]

array([ 0.04372703,  0.22827551,  0.15656076,  0.05237848, -0.05006586,
       -0.0986954 ,  0.15983543,  0.2943019 , -0.14251803, -0.06070546,
        0.5778855 ,  0.13686685,  0.08449788, -0.44681913,  0.01979469,
       -0.1957505 ,  0.26323652, -0.00643774, -0.482569  , -0.16440654,
       -0.03104678, -0.29329532,  0.10434166,  0.23916727,  0.06357735,
       -0.03900078, -0.0947843 ,  0.19263105, -0.15048622, -0.2338036 ,
       -0.01798677, -0.52638173,  0.33945128, -0.06111382,  0.29359928,
        0.17500456,  0.11996654, -0.3034498 , -0.36776918,  0.05197386,
       -0.05260934,  0.00476697, -0.22259073, -0.14970438,  0.17256969,
       -0.00386069, -0.08359666,  0.13787009, -0.11536425,  0.3996215 ,
       -0.15150256,  0.05159961, -0.05135211,  0.33399057,  0.26028588,
        0.24064623,  0.05033691,  0.05090418,  0.32481053, -0.4068299 ],
      dtype=float32)

In [33]:
# build a model
def init_model():
    model = Sequential()
    model.add(Masking())
    model.add(SimpleRNN(20, activation="tanh"))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(60, activation="linear"))

    model.compile(loss='mse',
                  optimizer='rmsprop',
                  metrics=['mae'])
    
    return model

model = init_model()

2022-06-27 17:21:53.252514: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-27 17:21:53.252554: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-27 17:21:53.252594: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (think): /proc/driver/nvidia/version does not exist
2022-06-27 17:21:53.252974: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [34]:
es = EarlyStopping(patience=3, restore_best_weights=True)

model.fit(X_train_pad, y_train_vec, 
          batch_size = 16,
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
264/264 [==============================] - 4s 13ms/step - loss: 0.1678 - mae: 0.2859 - val_loss: 0.1444 - val_mae: 0.2638
Epoch 2/100
264/264 [==============================] - 3s 10ms/step - loss: 0.1399 - mae: 0.2607 - val_loss: 0.1399 - val_mae: 0.2578
Epoch 3/100
264/264 [==============================] - 3s 10ms/step - loss: 0.1363 - mae: 0.2566 - val_loss: 0.1382 - val_mae: 0.2570
Epoch 4/100
264/264 [==============================] - 3s 10ms/step - loss: 0.1343 - mae: 0.2542 - val_loss: 0.1354 - val_mae: 0.2541
Epoch 5/100
264/264 [==============================] - 3s 10ms/step - loss: 0.1335 - mae: 0.2529 - val_loss: 0.1345 - val_mae: 0.2509
Epoch 6/100
264/264 [==============================] - 3s 10ms/step - loss: 0.1329 - mae: 0.2523 - val_loss: 0.1352 - val_mae: 0.2512
Epoch 7/100
264/264 [==============================] - 3s 10ms/step - loss: 0.1326 - mae: 0.2519 - val_loss: 0.1345 - val_mae: 0.2500
Epoch 8/100
264/264 [==============================] - 3s 10ms

In [35]:
model.evaluate(X_test_pad, y_test_vec)

81/81 [==============================] - 0s 4ms/step - loss: 0.1367 - mae: 0.2524


[0.1366543173789978, 0.2524428963661194]

In [36]:
X_test[0], y_test[0]

(['role',
  'with',
  'his',
  'trademark',
  'cock',
  'eyed',
  'virility',
  'hilliard',
  'is',
  'an',
  'extremely',
  'dull',
  'presence',
  'and',
  'as',
  'a',
  'former',
  'band',
  'singer',
  'she',
  'performs',
  'two',
  'berlin',
  'love',
  'songs',
  'in',
  'a',
  'frustratingly',
  'diffident',
  'manner',
  'regardless',
  'the',
  'magic',
  'generated',
  'by',
  'astaire',
  'and',
  'rogers',
  'in',
  'their'],
 'wants')

In [37]:
X_test_pad[:1, :, :].shape

(1, 40, 60)

In [38]:
y_pred = model.predict(X_test_pad[:1, :, :])
y_pred

1/1 [==============================] - 0s 171ms/step


array([[ 0.06051656,  0.35009402,  0.26440102,  0.04520664, -0.02902132,
        -0.17187405,  0.2417962 ,  0.33463684, -0.20373523, -0.15549763,
         0.8187187 ,  0.11520192,  0.04938477, -0.48560056, -0.07183513,
        -0.27102306,  0.42608008, -0.03075608, -0.6184177 , -0.19264162,
         0.03570593, -0.41185495,  0.20316753,  0.29543114,  0.10288557,
         0.07482363, -0.08707804,  0.20746544, -0.22759783, -0.39536488,
         0.00853563, -0.62692666,  0.43602815, -0.09800893,  0.4056929 ,
         0.14734806,  0.21196838, -0.46755174, -0.4296395 ,  0.01488025,
        -0.2460854 ,  0.04334975, -0.25841773, -0.33185482,  0.2269401 ,
         0.05472926, -0.10662878,  0.1440764 , -0.1782812 ,  0.6468891 ,
        -0.1706374 ,  0.03132308, -0.1428622 ,  0.5371611 ,  0.3648012 ,
         0.20618124, -0.07461578,  0.01789954,  0.25359443, -0.47009268]],
      dtype=float32)

In [39]:
y_pred[0].shape

(60,)

In [40]:
wv.similar_by_vector(y_pred[0])

[('climax', 0.9883348345756531),
 ('vampire', 0.9881560802459717),
 ('writer', 0.9880637526512146),
 ('utterly', 0.987765371799469),
 ('team', 0.9875910878181458),
 ('virgin', 0.9875379800796509),
 ('comedic', 0.9873790144920349),
 ('david', 0.9873227477073669),
 ('setting', 0.9867087602615356),
 ('french', 0.9864742755889893)]

In [41]:
# now we need to append y_pred to X_test[0] and predict over and over to predict multiple words

In [42]:
new_word = wv.similar_by_vector(y_pred[0], topn=1)[0][0]
new_word

'climax'

In [43]:
X_new = X_test[0][1:]
X_new.append(new_word)
X_new = [X_new]

In [44]:
len(X_new)

1

In [45]:
len(X_new[0]), len(X_test[0])

(40, 40)

In [46]:
X_new_embed = embedding(word2vec, X_new)
X_new_pad = pad_sequences(X_new_embed, dtype='float32', padding='post', maxlen=40)

In [47]:
X_new_pad.shape

(1, 40, 60)

In [48]:
y_pred = model.predict(X_new_pad)

1/1 [==============================] - 0s 30ms/step


In [49]:
wv.similar_by_vector(y_pred[0])

[('vampire', 0.9930422902107239),
 ('comedic', 0.9919933676719666),
 ('la', 0.9912380576133728),
 ('french', 0.9912087321281433),
 ('bizarre', 0.9909916520118713),
 ('deep', 0.9908912181854248),
 ('german', 0.9905884265899658),
 ('era', 0.9905694127082825),
 ('utterly', 0.9905291795730591),
 ('animation', 0.9904189705848694)]

In [50]:
# let's put this into a function
def repeat_prediction(sentence, repetition=10):
    new_sentence = sentence
    X = sentence.split()
    for i in range(repetition):
        X_embed = embedding(word2vec, [X])
        X_pad = pad_sequences(X_embed, dtype='float32', padding='post', maxlen=40)
        y_pred = model.predict(X_pad)
        new_word = wv.similar_by_vector(y_pred[0], topn=1)[0][0]
        X.pop(0)
        X.append(new_word)
        new_sentence += " " + new_word
    return new_sentence

sentence = "he doesn't realize that his behavior should change and continues to act as he had before he listens to rap music sings along and plays the stereotypical part of an urban black man the real humor in this"
new_sentence = repeat_prediction(sentence)
new_sentence

1/1 [==============================] - 0s 27ms/step


"he doesn't realize that his behavior should change and continues to act as he had before he listens to rap music sings along and plays the stereotypical part of an urban black man the real humor in this overall flick miss miss word extreme near awesome near awesome"

In [52]:
sentence = "i like this movie and the actor"
new_sentence = repeat_prediction(sentence)
new_sentence

1/1 [==============================] - 0s 31ms/step


'i like this movie and the actor visual near awesome near awesome near awesome near awesome near'